In [3]:
%matplotlib inline


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d, fftconvolve
from numpy.fft import rfft2, irfft2

In [5]:
from matplotlib import colors
import matplotlib.animation as animation
from IPython.display import HTML

# Numbers, Arrays and Life

Cellular automata are discrete-space, discrete-time models of spatio-temporal processes. In these models, signal propagation typically involves updating the state of each cell in a grid based on some function of its local neighbors. We will use a toy example, but the techniques you will implement generalize to useful contexts, e.g. spatial statistics or image processing.

We will use John Conway's [Game of Life](https://en.wikipedia.org/wiki/Conway's_Game_of_Life) to practice our manipulation of numerical arrays. In particular, we will see how to use the vectorization, views, convolution and Fourier transform capabilities of `numpy` to solve the same problem in different ways.

See Wikipedia for detailed rules. Here is a summary:

```
Any live cell with fewer than two live neighbors dies, as if caused by under-population.
Any live cell with two or three live neighbors lives on to the next generation.
Any live cell with more than three live neighbors dies, as if by over-population.
Any dead cell with exactly three live neighbors becomes a live cell, as if by reproduction.
```

There are two common ways to count neighbors for cells that are at the edge of the grid. With periodic boundary conditions, the grid wraps around (so it is a torus), that is, the coordinate -1 is mapped to the max_coord, and max_coord + 1 is mapped to 0. Alternatively, just consider any neighbor outside the grid to be equal to zero. For simplicity, we have chosen a life configuration where both strategies give the same results, so you can implement either strategy.

### Initial grid configuration

In [6]:
gun = np.load('gosper_gun.npy')
init_grid = np.zeros((64, 64)).astype('int')
for y, x in gun:
    init_grid[8+x, 8+y] = 1


### Utility function to show animation

In [7]:
def animate(i):
    """Function to display animation for iteration i."""
    global grid
    grid = step(grid)
    im.set_array(grid)
    return im,

**1**. Write a function to play one step in the Game of Life. The function must be named `step` and take a single argument `grid` which is a `numpy` array containing the initial configuration, and return a new `numpy` array containing the updated configuration. Use for loops to implement this first version of the step function.

In [8]:

def step(grid):
    """Function to play one step in the Game of Life
    Input: a numpy array
    Output: an upadted numpy array
    """
    #find the number of rows and columns in the grid
    num_rows=grid.shape[0]
    num_cols=grid.shape[1]

    #initialize new grid
    new_grid=grid.copy()
    #iterate through each position in the grid 
    for i in range (0,num_rows):
        for j in range(0,num_cols):
            
            #go through each of the posibilities of neighboring coordinates. 
            #if the neighboring coordinates goes off the grid, change to 0 or max_cooridinate, as appropriate
            above=i+1
            if above==num_rows:
                above=0 
            below=i-1
            if below<0:
                below=num_rows-1
            right=j+1
            if right==num_cols:
                right=0
            left=j-1
            if left<0:
                left=num_cols-1
                
            #Now go through each of the neighbors and count how many live neighbors there are 
            live_count=grid[i,left]+grid[i,right]+grid[above,j]+grid[below,j]+grid[below,right]+grid[below,left]+grid[above,right]+grid[above,left]            
            
            #after counting, set the position to live/dead based on live_count
            if grid[i,j]==0 and live_count==3:
                new_grid[i,j]=1
            if grid[i,j]==1:
                if live_count<2 or live_count>3:
                    new_grid[i,j]=0
             
    #after iterating through each position in the grid, return the updated grid
    return(new_grid)      

init_grid=step(init_grid)

In [9]:
step(init_grid)

fig = plt.figure(figsize=(5, 5))
grid = init_grid.copy()
im = plt.imshow(grid, animated=True, interpolation='nearest', cmap='gray')
plt.close(fig)

anim = animation.FuncAnimation(fig, animate, 
                               frames=60, interval=50, blit=True)
animation.rcParams['animation.writer'] = 'avconv'
HTML(anim.to_html5_video())


**2**. Rewrite the step function using vectorization. That is, use eight different views of the grid to calculate the neighbor sum instead of double for loops.

In [10]:

def step2(grid):
    '''Function to play one step in the Game of Life using vectorization
    This function considers any a neighbor outside of the grid to be zero
    Input: a numpy array
    Output: an upadted numpy array
    '''
    
    #get dimesions
    num_rows=grid.shape[0]
    num_cols=grid.shape[1]
    
    #create new grid of the same size, but all 0s
    neighbor_count=np.zeros(grid.shape)
    
    #add neighbor below
    neighbor_count[1:num_rows,:]=grid[0:num_rows-1,:]

    
     #add neighbor above
    neighbor_count[0:num_rows-1,:]=neighbor_count[0:num_rows-1,:]+grid[1:num_rows,:]
 
    #add neighbore to the right
    neighbor_count[:,1:num_cols]=neighbor_count[:,1:num_cols]+grid[:,0:num_cols-1]
    
    #add neighbor to the left
    neighbor_count[:,0:num_cols-1]=neighbor_count[:,0:num_cols-1]+grid[:,1:num_cols]
  
    #add add above and right
    neighbor_count[1:num_rows,1:num_cols]=neighbor_count[1:num_rows,1:num_cols]+grid[0:num_rows-1,0:num_cols-1]
 
    #add add above and left
    neighbor_count[1:num_rows,0:num_cols-1]=neighbor_count[1:num_rows,0:num_cols-1]+grid[0:num_rows-1,1:num_cols]

    #add below and right
    neighbor_count[0:num_rows-1,1:num_cols]=neighbor_count[0:num_rows-1,1:num_cols]+grid[1:num_rows,0:num_cols-1]
 
    #add below and left
    neighbor_count[0:num_rows-1,0:num_cols-1]=neighbor_count[0:num_rows-1,0:num_cols-1]+grid[1:num_rows,1:num_cols]
 
    ##### Update new grid using neighbor count ######
    #change 0 to negative one and then multipy the two matrices together
    #Now cells that were dead before will be negative and those that were alive will be positive
    #I use this pos/neg to update my grids.
    grid[grid==0]=-1
    mult=np.multiply(grid, neighbor_count)
    #make a new grid of zeros
    new_grid=np.zeros([num_rows,num_cols])
    
    #change to alive if previously alive and with correct number of neighbors
    new_grid[ mult==2]=1
    new_grid[ mult==3]=1
    
    #change to alive if previously dead and correct number of neighbors
    new_grid[mult==-3]=1
   
    
    #return the updated grid
    return(new_grid)   

init_grid=step2(init_grid)


In [11]:
fig = plt.figure(figsize=(5, 5))
grid = init_grid.copy()
im = plt.imshow(grid, animated=True, interpolation='nearest', cmap='gray')
plt.close(fig)

anim = animation.FuncAnimation(fig, animate, 
                               frames=60, interval=50, blit=True)
animation.rcParams['animation.writer'] = 'avconv'              
HTML(anim.to_html5_video())

**3a**. A discrete 2D convolution generates a weighted sum of a 2D grid, with the weights given by a 2D kernel. For example, the kernel

```
0 1 0
1 1 1
0 1 0
```

would result in summing the current location and the N, E, S, W neighbors.

Use the `convolve2d` function from `scipy.signal` (already imported for you in this notebook) to implement the 3rd version of the `step` function with a suitable kernel and with `mode=same` to preserve the grid size across iterations.

In [12]:
def step3(grid):
    
    '''Function to play one step in the Game of Life using convolve2d
    Note that in this function, if a cell is on the border of the grid, it's neighbor outside of the grid is counted as 0
    Input: a numpy array
    Output: an upadted numpy array
    
    '''
    
    num_rows=grid.shape[0]
    num_cols=grid.shape[1]
    
    #create a kernel, which is 3x3 of ones, except the middle which is 0
    kernel=np.ones([3,3])
    kernel[1,1]=0
    
    #use the kernel to sum up the neighbors
    neighbor_count=convolve2d(grid,kernel,mode='same')
    
   
     ##### Update new grid using neighbor count ######
    #change 0 to negative one and then multipy the two matrices together
    #Now cells that were dead before will be negative and those that were alive will be positive
    #I use this pos/neg to update my grids.
    grid[grid==0]=-1
    mult=np.multiply(grid, neighbor_count)
    
    #make a new grid of zeros
    new_grid=np.zeros([num_rows,num_cols])
    
    #change to alive if previously alive and with correct number of neighbors
    new_grid[ mult==2]=1
    new_grid[ mult==3]=1
    
    #change to alive if previously dead and correct number of neighbors
    new_grid[ mult==-3]=1
    #note that all cells that do not fit this criteria will be left as dead
  
    
    #return the new grid
    return(new_grid)

init_grid=step3(init_grid)



In [13]:
fig = plt.figure(figsize=(5, 5))
grid = init_grid.copy()
im = plt.imshow(grid, animated=True, interpolation='nearest', cmap='gray')
plt.close(fig)

anim = animation.FuncAnimation(fig, animate, 
                               frames=60, interval=50, blit=True)
animation.rcParams['animation.writer'] = 'avconv'              
HTML(anim.to_html5_video())

**3b**. One way to multiply two numbers that you are familiar with is to move to the log domain where addition is equivalent to multiplication. For example, instead of calculating $100 \times 1000$, we can calculate $\exp(\log(100) + \log(1000))$ to get the same result to roundoff error. In the same way, a convolution is equivalent to multiplication in the Fourier domain.  

Implement the fourth version of the `step` function using the `fftconvolve` function for real Fast Fourier Transforms from the `scipy.signal` package. Because of rounding errors, you need to round the results returned by `fftconvolve`.

In [14]:
def step4(grid):
    
    '''Function to play one step in the Game of Life using fftconvolve
    Note that in this function if a cell is at the border, it's neighbor outside of the grid is listed as zero
    Input: a numpy array
    Output: an upadted numpy array
    '''
    num_rows=grid.shape[0]
    num_cols=grid.shape[1]
    
    #create a kernel, which is 3x3 of ones, except the middle which is 0
    kernel=np.ones([3,3])
    kernel[1,1]=0
    
    #use the kernel to sum up the neighbors. Make sure to round
    neighbor_count=np.round(fftconvolve(grid,kernel,mode='same'))
    
    
     ##### Update new grid using neighbor count ######
    #change 0 to negative one and then multipy the two matrices together
    #Now cells that were dead before will be negative and those that were alive will be positive
    #I use this pos/neg to update my grids.
    
    grid[grid==0]=-1
    mult=np.multiply(grid, neighbor_count)
    
    #make a new grid of zeros
    new_grid=np.zeros([num_rows,num_cols])
    
    #change to alive if previously alive and with correct number of neighbors
    new_grid[ mult==2]=1
    new_grid[ mult==3]=1
    
    #change to alive if previously dead and correct number of neighbors
    new_grid[ mult==-3]=1
    #change the other parts of the grid to the correct values
    
    #return the new grid
    return(new_grid)

init_grid=step4(init_grid)




In [15]:
fig = plt.figure(figsize=(5, 5))
grid = init_grid.copy()
im = plt.imshow(grid, animated=True, interpolation='nearest', cmap='gray')
plt.close(fig)

anim = animation.FuncAnimation(fig, animate, 
                               frames=60, interval=50, blit=True)
HTML(anim.to_html5_video())

**4**. Modify 3a to model the [Forest Fire model](https://en.wikipedia.org/wiki/Forest-fire_model) instead of the Game of Life, with `f=0.001` and `p=0.1`.

In [27]:
import random as random

def forest_fire(grid):
    '''Function to carry out one step of Forest Fire. 
    Input: Grid. A zero corresponds to empty, one to occupied, and two to burning
    Output: Updated grid
    
    '''
    #parameteres
    f=.001
    p=.1
    
    #create a kernel, which is 3x3 of ones, except the middle which is 0
    kernel=np.ones([3,3])
    kernel[1,1]=0
    
    #find dimensions of grid
    num_rows=grid.shape[0]
    num_cols=grid.shape[1]
    
    #create new empty grid
    new_grid=np.zeros([num_rows,num_cols])
    
    #
    burning=np.zeros([num_rows,num_cols])
    burning[grid==2]=1

    #count number of neigbhors that are burning
    burning_count=convolve2d(burning,kernel,mode='same')

    
    #update new grid according to the rules of the game
    for i in range(0,num_rows):
        for j in range(0,num_cols):
            
            #if the oringinal grid is burning
            if grid[i,j]==2:
                new_grid[i,j]=0
            
            #if the original grid is occupied
            if grid[i,j]==1:
                if burning_count[i,j]>0:
                    new_grid[i,j]=2
                else:
                    prob1=random.random()
                    if prob1<=f:
                        print(prob1)
                        new_grid[i,j]=2 
            #if the original grid is empty
            if grid[i,j]==0:
                prob2=random.random()
                if prob2<=p:
                    new_grid[i,j]=1
                
    #return the updated grid            
    return(new_grid)

init_grid=forest_fire(init_grid)

0.00013302589686725597


In [28]:
cmap = colors.ListedColormap(['black', 'green', 'red'])

fig = plt.figure(figsize=(5, 5))
grid = init_grid.copy()
im = plt.imshow(grid, animated=True, interpolation='nearest', cmap=cmap, vmax=2)
plt.close(fig)

anim = animation.FuncAnimation(fig, animate, 
                               frames=100, interval=50, blit=True)
HTML(anim.to_html5_video())